In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.constraints import max_norm
import tensorflow.keras.regularizers as regulizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

import os
import numpy as np
import csv


TensorFlow 2.x selected.
2.1.0-rc1


In [ ]:
# !fusermount -u drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

root_path = 'drive/My Drive/duth-cv-2019-2020-hw-4/'
base_dir = root_path + 'vehicles/' # or '/kaggle/input/duth-cv-2019-2020-hw-4/vehicles'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

In [ ]:
# Some hyper-parameters:
img_height = 224 
img_width =  224 
input_shape = [img_height, img_width, 3] 
batch_size  = 128       
num_epochs  = 100           
lr = 1e-2
decay = 0.0 

# Number of classes:
num_classes = 6

In [ ]:
def _res_conv(filters, kernel_size=3, padding='same', strides=1, use_relu=True, use_bias=False, 
              kernel_initializer='random_uniform', kernel_regularizer=None, kernel_constraint=None, name = 'cbr'):
    """
    Create a residual convolutional block.

    This function returns a function that creates a residual convolutional block
    with the given parameters. The block consists of a 2D convolutional layer,
    followed by a batch normalization layer and an activation layer (if specified).

    Parameters
    ----------
    filters : int
        Number of filters in the convolutional layer.
    kernel_size : int, optional
        Size of the kernel for the convolutional layer, by default 3.
    padding : str, optional
        Padding to use in the convolutional layer, by default 'same'.
    strides : int, optional
        Strides to use in the convolutional layer, by default 1.
    use_relu : bool, optional
        Whether to use a ReLU activation after the batch normalization layer, by default True.
    use_bias : bool, optional
        Whether to use a bias term in the convolutional layer, by default False.
    kernel_initializer : str, optional
        Kernel initializer to use for the convolutional layer, by default 'random_uniform'.
    kernel_regularizer : keras regularizer, optional
        Regularization to use for the convolutional layer, by default None.
    kernel_constraint : keras constraint, optional
        Constraint to use for the convolutional layer, by default None.
    name : str, optional
        Name to use for the block, by default 'cbr'.

    Returns
    -------
    function
        A function that creates a residual convolutional block with the given parameters.
    """
    def layer_fn(x):
        # Convolutional layer
        conv = Conv2D(filters=filters, kernel_size=kernel_size, padding=padding, strides=strides, use_bias=use_bias,
                        kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer, kernel_constraint=kernel_constraint, name=name + '_c')(x)
        # Batch normalization layer
        act = BatchNormalization(axis=-1, name=name + '_bn')(conv)
        # Activation layer (if specified)
        if use_relu:
            act = Activation('relu', name=name + '_r')(act)
        return act
    return layer_fn

In [ ]:
def _residual_block(filters, kernel_size=3, strides=1, use_bias=False, 
                          kernel_initializer='random_uniform', kernel_regularizer=None, kernel_constraint=None, name='res_bl'):
    """
    Create a residual block.

    This function returns a function that creates a residual block with the given parameters. 
    The block consists of two residual convolutional blocks, with a shortcut connection between the input and output. 
    The first block has the specified strides, while the second block has a stride of 1.

    Parameters
    ----------
    filters : int
        Number of filters in the convolutional layers of the block.
    kernel_size : int, optional
        Size of the kernel for the convolutional layers, by default 3.
    strides : int, optional
        Strides to use in the first convolutional layer of the block, by default 1.
    use_bias : bool, optional
        Whether to use a bias term in the convolutional layers, by default False.
    kernel_initializer : str, optional
        Kernel initializer to use for the convolutional layers, by default 'random_uniform'.
    kernel_regularizer : keras regularizer, optional
        Regularization to use for the convolutional layers, by default None.
    kernel_constraint : keras constraint, optional
        Constraint to use for the convolutional layers, by default None.
    name : str, optional
        Name to use for the block, by default 'res_bl'.

    Returns
    -------
    function
        A function that creates a residual block with the given parameters.
    """
    def layer_fn(x): 
    
        # Residual Path:     
        x_conv1 = _res_conv(filters=filters, kernel_size=kernel_size, padding='same', strides=strides, use_relu = True, use_bias=use_bias, 
                            kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer, kernel_constraint=kernel_constraint, name=name + '_cbr_1')(x)
                            
        x_residual = _res_conv(filters=filters, kernel_size=kernel_size, padding='same', use_relu = False, strides=1, use_bias=use_bias,
                                kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer, kernel_constraint=kernel_constraint, name=name + '_cbr_2')(x_conv1)
                            
        # Shortcut Path:
        shortcut = x if strides == 1 else Conv2D(filters, kernel_size=1, padding='valid', strides=strides, 
                                                kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer, name=name + '_shortcut_c')(x)                                      

        return Activation('relu')(Add([shortcut, x_residual]))                      
    
    return layer_fn


In [ ]:
def _residual_macroblock(block_fn, filters, repetitions=2, kernel_size=3, strides_1st_block = 1, use_bias=False,
                            kernel_initializer='random_uniform', kernel_regularizer=None, kernel_constraint=None, name='res_macroblock'):
    """
    Create a residual macroblock.

    This function returns a function that creates a residual macroblock with the given parameters. 
    A macroblock is a sequence of residual blocks with the same number of filters. 
    This function takes in a block function, which is used to create each individual block within the macroblock.

    Parameters
    ----------
    block_fn : function
        A function that creates a residual block.
    filters : int
        Number of filters in the convolutional layers of the block.
    repetitions : int, optional
        Number of blocks to include in the macroblock, by default 2.
    kernel_size : int, optional
        Size of the kernel for the convolutional layers, by default 3.
    strides_1st_block : int, optional
        Strides to use in the first convolutional layer of the first block in the macroblock, by default 1.
    use_bias : bool, optional
        Whether to use a bias term in the convolutional layers, by default False.
    kernel_initializer : str, optional
        Kernel initializer to use for the convolutional layers, by default 'random_uniform'.
    kernel_regularizer : keras regularizer, optional
        Regularization to use for the convolutional layers, by default None.
    kernel_constraint : keras constraint, optional
        Constraint to use for the convolutional layers, by default None.
    name : str, optional
        Name to use for the macroblock, by default 'res_macroblock'.

    Returns
    -------
    function
        A function that creates a residual macroblock with the given parameters.
    """
    def layer_fn(x):
        for i in range(repetitions):
            block_name = "{}_{}".format(name, i) 
            strides = strides_1st_block if i == 0 else 1
            x = block_fn(filters=filters, kernel_size=kernel_size, strides=strides, use_bias=use_bias,
                            kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer, kernel_constraint=kernel_constraint, name=block_name)(x)
        return x
    return layer_fn

In [ ]:
def ResNet(input_shape, num_classes=num_classes, block_fn=_residual_block, repetitions=(2, 2, 2, 2),
            use_bias=False, kernel_initializer='random_uniform', kernel_regularizer=None, kernel_constraint=None):
    """
    Create a ResNet model.

    This function creates a ResNet model with the given parameters. 
    The model consists of an input layer, followed by four macroblocks with increasing number of filters, 
    and a final output layer.

    Parameters
    ----------
    input_shape : tuple
        Shape of the input layer.
    num_classes : int, optional
        Number of classes to classify, by default num_classes.
    block_fn : function, optional
        A function that creates a residual block, by default _residual_block.
    repetitions : tuple, optional
        Number of blocks in each macroblock, by default (2, 2, 2, 2).
    use_bias : bool, optional
        Whether to use a bias term in the convolutional layers, by default False.
    kernel_initializer : str, optional
        Kernel initializer to use for the convolutional layers, by default 'random_uniform'.
    kernel_regularizer : keras regularizer, optional
        Regularization to use for the convolutional layers, by default None.
    kernel_constraint : keras constraint, optional
        Constraint to use for the convolutional layers, by default None.

    Returns
    -------
    keras model
        A ResNet model with the given parameters.
    """
    # Input and 1st layers:
    inputs = Input(shape=input_shape)
    conv = _res_conv(filters=64, kernel_size=7,padding='same', strides=2, use_relu=True, use_bias=use_bias,
                     kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer, kernel_constraint=kernel_constraint)(inputs)
    maxpool = MaxPooling2D(pool_size=3, strides=2, padding='same')(conv)

    # Chain of residual blocks:
    filters = 64
    strides = 2
    res_block = maxpool
    for i, repet in enumerate(repetitions):
        # We do not further reduce the input size for the 1st block (max-pool applied just before):
        block_strides = strides if i != 0 else 1
        macroblock_name = "block_{}".format(i) 
        res_block = _residual_macroblock(
            block_fn=block_fn, repetitions=repet, name=macroblock_name,
            filters=filters, strides_1st_block=block_strides, use_bias=use_bias,
            kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer, kernel_constraint=kernel_constraint)(res_block)
        filters = min(filters * 2, 1024) # we limit to 1024 filters max

    # Final layers for prediction:
    res_spatial_dim = tf.keras.backend.int_shape(res_block)[1:3]
    avg_pool = AveragePooling2D(pool_size=res_spatial_dim, strides=1)(res_block)
    flatten = Flatten()(avg_pool)
    predictions = Dense(units=num_classes, kernel_initializer=kernel_initializer, activation='softmax')(flatten)

    # Model:
    model = Model(inputs=inputs, outputs=predictions)
    return model

In [ ]:
def MyResNet(input_shape, num_classes=num_classes, use_bias=True, 
             kernel_initializer='random_uniform', kernel_regularizer=None, kernel_constraint=max_norm(3)): 
    return ResNet(input_shape, num_classes, block_fn=_residual_block, repetitions=(2,2,2), 
                  use_bias=use_bias, kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer, kernel_constraint=kernel_constraint) 

In [ ]:
#model creation
model = MyResNet(input_shape=input_shape, num_classes=num_classes)

# Compile the model
model.compile(loss='categorical_crossentropy', 
              optimizer= tf.keras.optimizers.SGD(lr=lr, decay=decay, nesterov=True, momentum=0.9),
              metrics=['accuracy'])

model.summary()

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   horizontal_flip=True,
                                   rotation_range=10,  
                                   width_shift_range=.2, 
                                   height_shift_range=.2,
                                   zoom_range=0.1)
val_datagen  = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    target_size=(img_height,img_width),
                                                    shuffle=True)     

validation_generator =  val_datagen.flow_from_directory(validation_dir,
                                                        batch_size=batch_size,
                                                        class_mode='categorical',
                                                         target_size=(img_height,img_width)) 

Found 2494 images belonging to 6 classes.
Found 311 images belonging to 6 classes.


In [ ]:
import datetime

callbacks = []

logdir = os.path.join("/content/logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
callbacks.append(tensorboard_callback)

csv_logger = tf.keras.callbacks.CSVLogger('/content/logs/training.log')
callbacks.append(csv_logger)

save_best_callback = tf.keras.callbacks.ModelCheckpoint(f'small_last4.h5',monitor='val_accuracy', save_best_only=True) 
callbacks.append(save_best_callback)

def scheduler(epoch):
  if epoch < 10:
    return lr
  else:
    return lr * tf.math.exp(0.1 * (10 - epoch))

lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
callbacks.append(lr_callback)

earlystop = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', 
                          min_delta = 0.01, 
                          patience = 15, 
                          verbose = 1,
                          restore_best_weights = True)
callbacks.append(earlystop)


# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=num_epochs,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1,
      callbacks = callbacks)
 
# Save the model
model.save('small_last4_final.h5')

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 19.484375 steps, validate for 2.4296875 steps
Epoch 1/100
20/19 [==============================] - 43s 2s/step - loss: 1.5601 - accuracy: 0.3665 - val_loss: 2.2194 - val_accuracy: 0.1415
Epoch 2/100
20/19 [==============================] - 38s 2s/step - loss: 1.2257 - accuracy: 0.5245 - val_loss: 2.3571 - val_accuracy: 0.1415
Epoch 3/100
20/19 [==============================] - 38s 2s/step - loss: 1.1011 - accuracy: 0.5646 - val_loss: 2.6010 - val_accuracy: 0.1415
Epoch 4/100
20/19 [==============================] - 39s 2s/step - loss: 1.0399 - accuracy: 0.5878 - val_loss: 2.6514 - val_accuracy: 0.1415
Epoch 5/100
20/19 [==============================] - 39s 2s/step - loss: 0.9576 - accuracy: 0.6231 - val_loss: 2.5138 - val_accuracy: 0.1608
Epoch 6/100
20/19 [==============================] - 39s 2s/step - loss: 0.9322 - accuracy: 0.6411 - val_loss: 3.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir='/content/logs'

In [ ]:
''' Predictions
'''

model = tf.keras.models.load_model('small_last4.h5') #'/kaggle/working/small_last4.h5'
rowlist = [['Id', 'Category']]

for dirname, _, filenames in os.walk('drive/My Drive/duth-cv-2019-2020-hw-4/vehicles/test'): #'/kaggle/input/duth-cv-2019-2020-hw-4/vehicles/test'
    for filename in filenames:
        path = os.path.join(dirname, filename)
        img = image.load_img(path, target_size=(img_height, img_width), grayscale=False, interpolation='bilinear')
        
        x = image.img_to_array(img)
        x = x * (1/255)
        x = np.expand_dims(x, axis=0)
        
        classes_pred = model.predict(x)
        cls_pred = np.argmax(classes_pred)
        rowlist.append([filename, cls_pred])
        print(filename, cls_pred)
        with open('output.csv', 'w', newline='') as file: 
            writer = csv.writer(file)
            writer.writerows(rowlist)

1388826099_01505abea9.jpg 2
1389089819_143b71928e.jpg 2
1389100435_024f76d449.jpg 2
139539078_ebb3ae33ae.jpg 5
1397712539_8f45c650ea.jpg 5
1389127753_0c90cae3d8.jpg 2
1389527018_5cb27542ee.jpg 2
1393571729_a3ee0597f7.jpg 3
1389986966_2e05482189.jpg 4
1399804209_24dd4e9ed4.jpg 0
1394770822_608d9a52b2.jpg 4
1398487316_9ddb8077a6.jpg 1
140579971_4735dc8c0f.jpg 0
1431159166_adae38daf5.jpg 4
1410830620_2efe7370c2.jpg 2
140898286_c4423857bb.jpg 2
1413051171_e95f73eb7f.jpg 5
1433160275_8a2febefce.jpg 4
1420768666_d72d0c4481.jpg 2
1425867308_1877f5522f.jpg 5
142340864_9572d4a2a7.jpg 1
1436187294_ab22051514.jpg 4
142402524_30ca16ef75.jpg 1
1436374646_2a40d8f9af.jpg 0
1437477213_af53fd281e.jpg 5
147389938_bba3e1fbc7.jpg 1
144439295_d5bdc2e693.jpg 1
148113522_3b0c62dcf1.jpg 0
1444628852_3c6dcc72a7.jpg 5
145426559_22d4300522.jpg 5
145848224_3333ac0525.jpg 2
1481459812_916784c5dd.jpg 4
145922523_829549e121.jpg 2
1481470556_e3e5a7b95e.jpg 4
1471727512_ada7337bdd.jpg 2
147311655_39288232f3.jpg 1
1490